## Part 1: Preprocessing

In [1]:
!pip install tensorflow

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'DistanceFromHome', 'Education', 'JobSatisfaction',
    'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
    'YearsSinceLastPromotion', 'NumCompaniesWorked'
]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes

,0
Age,int64
DistanceFromHome,int64
Education,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the data
X_train_ohe = ohe.fit_transform(X_train[["OverTime"]])

#Transform test data
X_test_ohe = ohe.transform(X_test[["OverTime"]])

# Get columns names for the OneHot features
ohe_columns = ohe.get_feature_names_out(["OverTime"])

# Create a DataFrame with the OneHot features
X_train_ohe_df = pd.DataFrame(X_train_ohe, columns=ohe_columns, index=X_train.index)
X_test_ohe_df = pd.DataFrame(X_test_ohe, columns=ohe_columns, index=X_test.index)

# Drop original column and add encoded columns
X_train_encoded = X_train.drop(columns=["OverTime"]).join(X_train_ohe_df)
X_test_encoded = X_test.drop(columns=["OverTime"]).join(X_test_ohe_df)

In [8]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train_encoded)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test_encoded)

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
y_dept_train = dept_encoder.fit_transform(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_dept_test = dept_encoder.transform(y_test[["Department"]])
y_dept_train.shape, y_dept_test.shape

((1102, 3), (368, 3))

In [10]:
# Create a OneHotEncoder for the Attrition column

attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
y_attr_train = attrition_encoder.fit_transform(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_attr_test = attrition_encoder.transform(y_test[["Attrition"]])
y_attr_train.shape, y_attr_test.shape

((1102, 2), (368, 2))

## Part 2: Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data.
input_dim = X_train_scaled.shape[1]
#print("Input dimension:", input_dim)

# Create the input layer
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
input_layer = Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
dept_output = Dense(3, activation='softmax', name='department_output')(dept_hidden)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attr_output = Dense(2, activation='softmax', name='attrition_output')(attr_hidden)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'categorical_crossentropy'
    },
    metrics={
        'department_output' : ['accuracy'],
        'attrition_output' : ['accuracy']
    }
)

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        768 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,989 (15.58 KB)

 Trainable params: 3,989 (15.58 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_dept_train, 'attrition_output': y_attr_train},
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - attrition_output_accuracy: 0.8662 - attrition_output_loss: 0.3213 - department_output_accuracy: 0.6601 - department_output_loss: 0.7219 - loss: 1.0435 - val_attrition_output_accuracy: 0.8416 - val_attrition_output_loss: 0.4187 - val_department_output_accuracy: 0.6063 - val_department_output_loss: 0.8485 - val_loss: 1.2711
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8878 - attrition_output_loss: 0.2972 - department_output_accuracy: 0.6659 - department_output_loss: 0.7216 - loss: 1.0187 - val_attrition_output_accuracy: 0.8416 - val_attrition_output_loss: 0.4133 - val_department_output_accuracy: 0.6018 - val_department_output_loss: 0.8511 - val_loss: 1.2683
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8681 - attrition_output_loss: 0.3329 - department_output_accuracy: 0.6716 - department_output_loss: 0.7072 - loss: 1.0399 - val_attrition_output_accuracy: 0.8281 - va

In [19]:
# Evaluate the model with the testing data
eval_results = model.evaluate(
    X_test_scaled,
    {
        'department_output': y_dept_test,
        'attrition_output': y_attr_test
    }
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8311 - attrition_output_loss: 0.5190 - department_output_accuracy: 0.5810 - department_output_loss: 1.1034 - loss: 1.6217 


In [20]:
# Print the accuracy for both department and attrition
print(f"Total Loss: {eval_results[0]:.4f}")
print(f"Department Loss: {eval_results[1]:.4f}")
print(f"Attrition Loss: {eval_results[2]:.4f}")
print(f"Department Accuracy: {eval_results[3]:.4f}")
print(f"Attrition Accuracy: {eval_results[4]:.4f}")

Total Loss: 1.5572
Department Loss: 1.0629
Attrition Loss: 0.4991
Department Accuracy: 0.8505
Attrition Accuracy: 0.5924


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is useful but may not be the best standalone metric, especially for the attrition prediction. If the dataset is imbalanced (e.g., many more “No Attrition” than “Yes”), accuracy can be misleading. In such cases, precision, recall, or F1-score might give a better picture of how well the model handles the minority class. For department prediction, accuracy is more reliable because it’s a multi-class problem and class balance may vary by dataset.
2. I used softmax activation for both output layers. Softmax is ideal for classification problems with mutually exclusive classes and one-hot encoded labels. It turns the output into a probability distribution across the classes, which works well for both the multi-class department prediction and the binary attrition prediction (which was also one-hot encoded into two columns).
3. Yes. Some ways to improve the model include:
	•	Performing hyperparameter tuning (adjusting layer sizes, number of layers, batch size, learning rate, etc.)
	•	Adding dropout or regularization to prevent overfitting
	•	Using class weights or resampling to handle potential class imbalance
	•	Running more training epochs while using early stopping to halt training when validation loss stops improving
	•	Evaluating with additional metrics like precision and recall, especially for attrition